<a href="https://colab.research.google.com/github/georgyzaouk/BS-Capstone/blob/main/smiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pubchempy pandas openpyxl
!pip install rdkit
!pip install openpyxl

In [6]:
# Imports
import pandas as pd
from pubchempy import get_compounds
from rdkit import Chem
import time
import requests

# Load datasets
LiverTox = pd.read_excel("/content/preLiverTox.xlsx")
DILIrank = pd.read_excel("/content/preDILIrank.xlsx")

# Validate SMILES
def is_valid_smiles(smiles):
    return Chem.MolFromSmiles(smiles) is not None

# Retrieve SMILES + CID from PubChemPy
def get_smiles(drug_name, max_retries=3):
    for attempt in range(max_retries):
        try:
            compounds = get_compounds(drug_name, 'name')
            if compounds:
                compound = compounds[0]
                smiles = compound.canonical_smiles
                cid = compound.cid
                if is_valid_smiles(smiles):
                    return smiles, cid
                else:
                    return None, cid  # CID found, but SMILES invalid
            return None, None  # Nothing found
        except Exception as e:
            if 'ServerBusy' in str(e) and attempt < max_retries - 1:
                time.sleep(2)
            else:
                return None, None

# Resolve SMILES + CID for LiverTox
def resolve_smiles_livertox(row):
    name1 = str(row['Ingredient'])
    name2 = str(row['Brand Name'])
    smiles, cid = get_smiles(name1)
    if not smiles and not cid:
        smiles, cid = get_smiles(name2)
    if not smiles:
        print(f"[⚠️ Not Found] {name1} / {name2}")
    return pd.Series({'SMILES': smiles, 'CID': cid})

# Resolve SMILES + CID for DILIrank
def resolve_smiles_dilirank(row):
    name = str(row['Compound Name'])
    smiles, cid = get_smiles(name)
    if not smiles:
        print(f"[⚠️ Not Found] {name}")
    return pd.Series({'SMILES': smiles, 'CID': cid})

# Start resolution
print("\n🔄 Resolving SMILES for LiverTox...")
LiverTox[['SMILES', 'CID']] = LiverTox.apply(resolve_smiles_livertox, axis=1)

print("\n🔄 Resolving SMILES for DILIrank...")
DILIrank[['SMILES', 'CID']] = DILIrank.apply(resolve_smiles_dilirank, axis=1)

# Save all data (including missing)
LiverTox.to_excel("/content/LiverToxSmiles.xlsx", index=False, engine='openpyxl')
DILIrank.to_excel("/content/DILIrankSmiles.xlsx", index=False, engine='openpyxl')

# Prepare missing SMILES report
missing_livertox = LiverTox[LiverTox['SMILES'].isna()][['Ingredient', 'Brand Name']].copy()
missing_dilirank = DILIrank[DILIrank['SMILES'].isna()][['Compound Name']].copy()

missing_livertox['Source'] = 'LiverTox'
missing_dilirank['Source'] = 'DILIrank'

missing_livertox['Name'] = missing_livertox['Ingredient'].fillna('') + " / " + missing_livertox['Brand Name'].fillna('')
missing_dilirank['Name'] = missing_dilirank['Compound Name']

missing_combined = pd.concat([
    missing_livertox[['Name', 'Source']],
    missing_dilirank[['Name', 'Source']]
], ignore_index=True)

missing_combined.to_excel("/content/MissingSmiles.xlsx", index=False)

# ✅ Final Report
print("\n✅ SMILES extraction complete!")
print(f"📉 Missing SMILES count:")
print(f" - LiverTox: {missing_livertox.shape[0]}")
print(f" - DILIrank: {missing_dilirank.shape[0]}")
print("📁 All compounds + CIDs saved in: /content/LiverToxSmiles.xlsx and /content/DILIrankSmiles.xlsx")
print("📁 Missing compounds saved in: /content/MissingSmiles.xlsx")



🔄 Resolving SMILES for LiverTox...
[⚠️ Not Found] Abatacept / Orencia
[⚠️ Not Found] Adalimumab / Humira
[⚠️ Not Found] Aducanumab / Aduhelm
[⚠️ Not Found] Aflibercept / Eylea
[⚠️ Not Found] Alemtuzumab / Campath
[⚠️ Not Found] Alglucerase / Ceredase
[⚠️ Not Found] Alglucosidase alfa / Lumizyme
[⚠️ Not Found] Alirocumab / Praluent
[⚠️ Not Found] Alpha-1 proteinase / Glassia
[⚠️ Not Found] Anakinra / Kineret
[⚠️ Not Found] Androgenic steroids / Group name
[⚠️ Not Found] Anifrolumab / Saphnelo
[⚠️ Not Found] Asparaginase / Eispar
[⚠️ Not Found] Atezolizumab / Tecentriq
[⚠️ Not Found] Atoltivimab / Inmazeb
[⚠️ Not Found] Avacincaptad pegol / Izervay
[⚠️ Not Found] Avelumab / Bavencio
[⚠️ Not Found] Axatilimab / Niktimvo
[⚠️ Not Found] Bamlanivimab / LY-CoV555
[⚠️ Not Found] Basiliximab / Simulect
[⚠️ Not Found] Belantamab-Mafodotin / Blenrep
[⚠️ Not Found] Belimumab / Benlysta
[⚠️ Not Found] Benralizumab / Fasenra
[⚠️ Not Found] Bevacizumab / Avastin
[⚠️ Not Found] Bezlotoxumab / Zinplav